# Extracting Files from a Zip Archive



1. **Import Modules:**
   - Imports the `zipfile` module to work with zip archives.
   - Imports the `os` module for interacting with the file system.

2. **Define Paths:**
   - `zip_file_path` is the path to the zip file you want to extract. (Adjust this as needed.)
   - `extract_dir` is the directory where the extracted files will be stored.

3. **Create Extraction Directory:**
   - Uses `os.makedirs()` with `exist_ok=True` to ensure the extraction directory exists (it creates the directory if it doesn't).

4. **Extract the Zip File:**
   - Opens the zip file in read mode using a context manager.
   - Extracts all the contents of the zip file into the specified extraction directory using `extractall()`.

In [2]:
import zipfile
import os

# Path to the zip file (adjust accordingly)
zip_file_path = "bttai-ajl-2025.zip"

# Directory where you want to extract the contents
extract_dir = "extracted_files"

# Create the directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Open the zip file and extract all contents
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to: {extract_dir}")

Files extracted to: extracted_files


# Loading and Preprocessing the Skin Disease Dataset

This code snippet sets up the environment for training a model on a skin disease dataset. It performs the following steps:

1. **Import Libraries:**
   - `torch` for deep learning operations.
   - `pandas` for data manipulation.
   - `os` for file path operations.
   - `torchvision.transforms` for image pre-processing and augmentation.
   - `torch.utils.data.Dataset` and `DataLoader` for creating and handling datasets.
   - `PIL.Image` for image processing.
   - `sklearn.utils.class_weight` for computing class weights (if needed).
   - `numpy` for numerical operations.

2. **Set Device:**
   - Checks if a CUDA-enabled GPU is available. If yes, it uses the GPU; otherwise, it falls back to the CPU.
   - Prints the device being used.

3. **Define Dataset Paths:**
   - `DATASET_FOLDER` is the root folder containing the dataset.
   - `TRAIN_IMG_DIR` specifies the directory where training images are stored.
   - `TRAIN_CSV` is the path to the CSV file that contains image metadata and labels.

4. **Define Image Transformations:**
   - Resizes images to 224x224 pixels.
   - Applies random horizontal flipping and random rotation (up to 10 degrees) for data augmentation.
   - Adjusts brightness and contrast with `ColorJitter`.
   - Converts images to PyTorch tensors.
   - Normalizes the images with a mean and standard deviation of 0.5 across all channels.

5. **Custom Dataset Class (`SkinDiseaseDataset`):**
   - Reads the CSV file using pandas to get image metadata and labels.
   - Creates a mapping from original text labels to numeric indices.
   - In the `__getitem__` method:
     - Constructs the image filename using the `md5hash` field.
     - Builds the full path to the image file, which is organized by label directories.
     - Opens the image and converts it to RGB.
     - Applies the defined transformations.
     - Returns the transformed image and its encoded label as a tensor.

6. **Load Dataset and Create DataLoader:**
   - Instantiates the `SkinDiseaseDataset` with the CSV path, image directory, and transformation pipeline.
   - Creates a `DataLoader` with a specified batch size (32) and shuffling enabled to iterate over the dataset during training.


In [4]:
import torch
import pandas as pd
import os
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Dataset paths
DATASET_FOLDER = "extracted_files"
TRAIN_IMG_DIR = os.path.join(DATASET_FOLDER, "train", "train")
TRAIN_CSV = os.path.join(DATASET_FOLDER, "train.csv")

# Define Transformations (224x224 + Data Augmentation)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

# Custom Dataset
class SkinDiseaseDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform
        self.label_mapping = {label: idx for idx, label in enumerate(sorted(self.data['label'].unique()))}
        self.data['label_encoded'] = self.data['label'].map(self.label_mapping)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = self.data.iloc[idx]['md5hash'] + ".jpg"
        label = torch.tensor(self.data.iloc[idx]['label_encoded'], dtype=torch.long)
        img_path = os.path.join(self.img_dir, self.data.iloc[idx]['label'], img_name)

        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)

        return image, label

# Load Dataset
train_dataset = SkinDiseaseDataset(TRAIN_CSV, TRAIN_IMG_DIR, transform=transform)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


Using device: cpu


# Selecting the Compute Device

This section determines the appropriate compute device based on your system's hardware capabilities:

1. **Check for Mac M1/M2 (MPS) Support:**
   - Uses `torch.backends.mps.is_available()` to check if the Metal Performance Shaders (MPS) backend is available.
   - Sets the device to `"mps"` to leverage the Metal API on Mac M1/M2 systems.

2. **Check for CUDA Availability:**
   - If MPS is not available, it checks whether a CUDA-capable GPU is available with `torch.cuda.is_available()`.
   - If available, sets the device to `"cuda"`.

3. **Default to CPU:**
   - If neither MPS nor CUDA is available, defaults to using the CPU.

4. **Print the Selected Device:**
   - Displays the selected device to confirm which hardware will be used for computations.


In [6]:
# Set device for Mac M1/M2 (MPS) or CUDA
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Use Metal API on Mac M1/M2
elif torch.cuda.is_available():
    device = torch.device("cuda")  # Use CUDA if available
else:
    device = torch.device("cpu")  # Default to CPU

print("Using device:", device)


Using device: mps



# Fine-tuning a Pre-trained ResNet-50 Model

This section prepares a pre-trained ResNet-50 model for transfer learning on your custom skin disease dataset:

1. **Import Required Modules:**
   - Imports `torch.nn` for defining neural network layers.
   - Imports `torchvision.models` to load pre-trained models.

2. **Load Pre-trained Model:**
   - Loads the ResNet-50 model with weights pre-trained on the ImageNet dataset using the updated PyTorch 2.0+ syntax.

3. **Freeze Pre-trained Layers:**
   - Iterates through all parameters in the model and disables gradient updates (i.e., `requires_grad = False`) to preserve learned features and speed up training.

4. **Modify the Final Classification Layer:**
   - Retrieves the number of input features from the original fully connected layer.
   - Replaces the final layer with a new `nn.Linear` layer configured to output a number of classes equal to the length of `train_dataset.label_mapping`.

5. **Device Configuration and Data Type Fix:**
   - Moves the model to the selected device (MPS, CUDA, or CPU) for computation.
   - Converts the model to `float32` to address potential data type issues on the MPS backend.
```

In [8]:
import torch.nn as nn
import torchvision.models as models

# Load Pre-trained ResNet-50 Model
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)  # Updated syntax for PyTorch 2.0+

# Freeze all layers except the last fully connected layer (speeds up training)
for param in model.parameters():
    param.requires_grad = False

# Modify the Final Classification Layer (to match your dataset's classes)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_dataset.label_mapping))  # New classification layer

# Move Model to MPS & Convert to float32
model = model.to(device)
model = model.to(torch.float32)  # Fix MPS dtype issue



# Handling Imbalanced Data with Class Weights

This code snippet addresses class imbalance in the dataset by:

1. **Extracting Labels:**
   - Retrieves the label values from the dataset's CSV.

2. **Computing Class Weights:**
   - Uses `compute_class_weight` with the `"balanced"` option from scikit-learn.
   - Calculates weights for each class based on their frequency in the dataset.

3. **Preparing Weights for PyTorch:**
   - Converts the computed class weights to a PyTorch tensor with type `float`.
   - Moves the tensor to the selected computation device (MPS, CUDA, or CPU).

4. **Defining a Weighted Loss Function:**
   - Sets up `nn.CrossEntropyLoss` with the class weights.
   - This helps to penalize errors on minority classes more during training, thus handling imbalance.
```

In [56]:
# Compute Class Weights to Handle Imbalanced Data
labels = train_dataset.data["label"].values
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Define Loss Function with Weights
criterion = nn.CrossEntropyLoss(weight=class_weights)


In [58]:
# Compute Class Weights to Handle Imbalanced Data
labels = train_dataset.data["label"].values
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

# Define Loss Function with Weights
criterion = nn.CrossEntropyLoss(weight=class_weights)


# Configuring the Optimizer and Learning Rate Scheduler

This section sets up the training optimization process:

1. **Optimizer Setup:**
   - **AdamW Optimizer:**  
     Uses `AdamW`, a variant of the Adam optimizer that includes weight decay for better regularization.
   - **Parameters:**  
     Applies the optimizer to all model parameters with:
     - A learning rate of 0.0005.
     - A weight decay of 1e-4 to prevent overfitting.

2. **Learning Rate Scheduler:**
   - **StepLR Scheduler:**  
     Reduces the learning rate by a factor of 0.5 (`gamma=0.5`) every 10 epochs (`step_size=10`).
   - **Purpose:**  
     Gradually lowers the learning rate during training, which helps in fine-tuning the model and stabilizing convergence.


In [60]:
import torch.optim as optim

# Use AdamW Optimizer for Better Regularization
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-4)

# Learning Rate Scheduler to Reduce LR Over Time
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)


# Training Loop Overview

This code block trains the model over 50 epochs. Below is a breakdown of the process:

- **Number of Epochs:**  
  - `num_epochs = 50` sets the training to run for 50 complete passes over the dataset.

- **Epoch Loop:**  
  - For each epoch, the model is set to training mode using `model.train()`.
  - A `running_loss` variable is initialized to accumulate the loss over all batches in the current epoch.

- **Batch Processing:**  
  - Iterates over batches from the `train_loader`.
  - Moves the input images and labels to the selected device (GPU/MPS/CPU).
  - Ensures that images are cast to `float32` and labels to `long` type.
  - Clears previous gradients with `optimizer.zero_grad()`.
  - Computes model outputs by passing the images through the model.
  - Calculates the loss using the defined criterion (e.g., weighted CrossEntropyLoss).
  - Performs backpropagation with `loss.backward()`.
  - Updates model parameters using `optimizer.step()`.
  - Accumulates the batch loss into `running_loss`.

- **Learning Rate Scheduling:**  
  - After processing all batches in an epoch, the scheduler updates the learning rate with `scheduler.step()`, gradually reducing it over time.

- **Progress Reporting:**  
  - Prints the average loss for each epoch to monitor training progress.

- **Training Completion:**  
  - After all epochs are completed, a message `"Training Complete!"` is printed to indicate that the training process is finished.


In [62]:
num_epochs = 50  # Increased for better learning

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        images = images.float()  # Convert images to float32
        labels = labels.long()   # Ensure labels are in long format

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

print("Training Complete!")



Epoch 1/50, Loss: 2.8618
Epoch 2/50, Loss: 2.3967
Epoch 3/50, Loss: 2.1586
Epoch 4/50, Loss: 2.0221
Epoch 5/50, Loss: 1.9227
Epoch 6/50, Loss: 1.8360
Epoch 7/50, Loss: 1.7422
Epoch 8/50, Loss: 1.6961
Epoch 9/50, Loss: 1.6638
Epoch 10/50, Loss: 1.6083
Epoch 11/50, Loss: 1.5519
Epoch 12/50, Loss: 1.5317
Epoch 13/50, Loss: 1.5082
Epoch 14/50, Loss: 1.4865
Epoch 15/50, Loss: 1.4624
Epoch 16/50, Loss: 1.4831
Epoch 17/50, Loss: 1.4622
Epoch 18/50, Loss: 1.4376
Epoch 19/50, Loss: 1.4265
Epoch 20/50, Loss: 1.4297
Epoch 21/50, Loss: 1.3718
Epoch 22/50, Loss: 1.3979
Epoch 23/50, Loss: 1.3819
Epoch 24/50, Loss: 1.3642
Epoch 25/50, Loss: 1.3594
Epoch 26/50, Loss: 1.3536
Epoch 27/50, Loss: 1.3416
Epoch 28/50, Loss: 1.3572
Epoch 29/50, Loss: 1.3252
Epoch 30/50, Loss: 1.3251
Epoch 31/50, Loss: 1.3301
Epoch 32/50, Loss: 1.3339
Epoch 33/50, Loss: 1.3142
Epoch 34/50, Loss: 1.3075
Epoch 35/50, Loss: 1.3177
Epoch 36/50, Loss: 1.3310
Epoch 37/50, Loss: 1.3017
Epoch 38/50, Loss: 1.3172
Epoch 39/50, Loss: 1.

# Evaluating Model Performance

This section evaluates the trained model using the accuracy metric:

1. **Set Evaluation Mode:**
   - `model.eval()` switches the model to evaluation mode, which disables dropout and batch normalization updates.

2. **Disable Gradient Calculations:**
   - The `torch.no_grad()` context prevents gradient computation, reducing memory usage and speeding up inference.

3. **Collect Predictions and Labels:**
   - Iterates over the data from `train_loader`.
   - Moves images and labels to the selected device.
   - Computes the model's outputs for the images.
   - Uses `torch.argmax` to select the class with the highest predicted score.
   - Accumulates predictions and true labels into `all_preds` and `all_labels` lists, converting them to NumPy arrays on the CPU.

4. **Calculate Accuracy:**
   - Uses `accuracy_score` from scikit-learn to compute the accuracy by comparing true labels with predictions.
   - Prints the accuracy percentage.


In [64]:
from sklearn.metrics import accuracy_score

model.eval()  # Set model to evaluation mode
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute Accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Model Accuracy: {accuracy * 100:.2f}%")


Model Accuracy: 59.69%


# Fine-tuning Pre-trained ResNet-50 with Partial Layer Freezing

This code snippet sets up a fine-tuning pipeline using a pre-trained ResNet-50 model with the following key steps:

1. **Import Required Modules:**
   - Imports `torch.nn` for building neural network components.
   - Imports `torchvision.models` to access pre-trained models.

2. **Load the Pre-trained Model:**
   - Loads the ResNet-50 model using weights pre-trained on the ImageNet dataset.  
   - The syntax used (`weights=models.ResNet50_Weights.IMAGENET1K_V1`) is updated for PyTorch 2.0+.

3. **Freeze Early Layers:**
   - Freezes all layers except for the last two blocks by iterating over the parameters.
   - The line `for param in list(model.parameters())[:-10]:` disables gradient updates (by setting `requires_grad = False`) for all parameters except the last 10, allowing the last few layers to be fine-tuned on the new dataset.

4. **Modify the Final Classification Layer:**
   - Retrieves the number of input features to the original fully connected layer (`model.fc.in_features`).
   - Replaces the final fully connected layer with a new one (`nn.Linear`) that matches the number of classes in your dataset, as defined by `len(train_dataset.label_mapping)`.

5. **Move the Model to the Device:**
   - Transfers the model to the selected computation device (e.g., MPS, CUDA, or CPU) for training.


In [68]:
import torch.nn as nn
import torchvision.models as models

# Load Pre-trained ResNet-50 Model
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)  # Updated for PyTorch 2.0+

# Freeze all layers EXCEPT last 2 blocks (to fine-tune)
for param in list(model.parameters())[:-10]:  # Freeze early layers
    param.requires_grad = False

# Modify the Final Classification Layer (Adjust for Your Dataset)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(train_dataset.label_mapping))  # New classification layer

# Move Model to Device
model = model.to(device)


# Image Transformations for ResNet-50

This transformation pipeline prepares the images for use with a ResNet-50 model by performing the following steps:

1. **Resize:**  
   - `transforms.Resize((224, 224))` adjusts the image size to 224x224 pixels, matching the standard input size for ResNet-50.

2. **Random Horizontal Flip:**  
   - `transforms.RandomHorizontalFlip()` randomly flips the image horizontally, which augments the dataset and helps the model generalize better.

3. **Random Rotation:**  
   - `transforms.RandomRotation(10)` rotates the image randomly by up to 10 degrees, introducing rotational variance.

4. **Color Jitter:**  
   - `transforms.ColorJitter(brightness=0.2, contrast=0.2)` randomly alters the brightness and contrast to simulate different lighting conditions.

5. **Random Affine Transformation:**  
   - `transforms.RandomAffine(degrees=15, translate=(0.1, 0.1))` applies random affine transformations including rotations up to 15 degrees and translations up to 10% of the image dimensions to simulate distortions.

6. **Conversion to Tensor:**  
   - `transforms.ToTensor()` converts the processed image into a PyTorch tensor, making it compatible with the model.

7. **Normalization:**  
   - `transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])` normalizes the tensor by centering the values around zero with a standard deviation of 0.5 for each channel, which helps in faster and more stable training.


In [70]:
from torchvision import transforms

# Image Transformations (ResNet-50 friendly)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),   # Randomly flip images
    transforms.RandomRotation(10),       # Rotate images slightly
    transforms.ColorJitter(brightness=0.2, contrast=0.2),  # Color variations
    transforms.RandomAffine(degrees=15, translate=(0.1, 0.1)),  # Simulated distortions
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize
])


# DataLoader with Weighted Sampling for Class Imbalance

This code snippet addresses class imbalance by computing class weights and incorporating them into a weighted sampler. This ensures that minority classes are sampled more frequently during training, leading to a more balanced learning process.

1. **Compute Class Weights:**
   - The labels from the dataset are first mapped to their numerical indices using `train_dataset.label_mapping`.
   - `compute_class_weight` from scikit-learn calculates weights for each class based on the "balanced" mode, which assigns higher weights to classes with fewer samples.

2. **Create Sample Weights:**
   - For every label in the dataset, a corresponding weight is assigned from the computed `class_weights` array.
   - This results in an array (`sample_weights`) where each sample’s weight reflects the importance of its class.

3. **Define Weighted Sampler:**
   - `WeightedRandomSampler` is created using the `sample_weights`. 
   - `num_samples` is set to the total number of samples, ensuring each sample is considered.
   - `replacement=True` allows a sample to be drawn multiple times, which is useful when the dataset is highly imbalanced.

4. **DataLoader with Weighted Sampling:**
   - The `DataLoader` is configured with the weighted sampler instead of a simple shuffle.
   - This setup ensures that during each epoch, the model sees a balanced representation of classes, improving the learning process for imbalanced datasets.


In [72]:
from torch.utils.data import WeightedRandomSampler
import numpy as np

# Compute Class Weights (Balance Data)
labels = train_dataset.data["label"].map(train_dataset.label_mapping).values
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
sample_weights = np.array([class_weights[label] for label in labels])

# Define Weighted Sampler
sampler = WeightedRandomSampler(weights=sample_weights, num_samples=len(sample_weights), replacement=True)

# DataLoader with Weighted Sampling
train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)


# Defining a Weighted Loss Function

This code snippet creates a loss function that accounts for class imbalance by applying weights to each class:

1. **Weighted Cross-Entropy Loss:**
   - `nn.CrossEntropyLoss` is used to calculate the loss between the model's predictions and the true labels.
   - The `weight` parameter is set to a tensor of class weights, ensuring that the loss penalizes misclassifications on under-represented classes more heavily.

2. **Tensor Conversion and Device Transfer:**
   - `torch.tensor(class_weights, dtype=torch.float)` converts the computed class weights into a PyTorch tensor of type `float`.
   - `.to(device)` moves this tensor to the selected computation device (e.g., MPS, CUDA, or CPU) to match the model's location.

This setup helps in training the model more effectively on imbalanced datasets by emphasizing the learning of minority classes.


In [74]:
criterion = nn.CrossEntropyLoss(weight=torch.tensor(class_weights, dtype=torch.float).to(device))


# Optimizer and Learning Rate Scheduler Setup

This code snippet configures the optimization strategy for training the model:

1. **AdamW Optimizer for Regularization:**
   - Uses `optim.AdamW` to optimize the model's parameters.
   - A learning rate of `0.001` is set along with a weight decay of `1e-4` to improve generalization by reducing overfitting.

2. **Cosine Annealing Learning Rate Scheduler:**
   - Applies `optim.lr_scheduler.CosineAnnealingLR` to adjust the learning rate during training.
   - The scheduler uses a cosine annealing schedule with `T_max=20`, meaning the learning rate will cycle over 20 epochs (or iterations), gradually reducing to help the model converge smoothly.


In [76]:
import torch.optim as optim

# Use AdamW Optimizer for Better Regularization
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=1e-4)

# Cosine Annealing for Learning Rate Adjustment
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=20)


# Training Loop with Gradient Accumulation

This code block trains the model over 50 epochs with gradient accumulation, which allows you to effectively increase the batch size without using more memory.

1. **Hyperparameters:**
   - `num_epochs = 50`: The total number of epochs for training.
   - `grad_accum_steps = 2`: Gradients are accumulated over 2 batches before performing an optimizer step.

2. **Epoch Loop:**
   - For each epoch, the model is set to training mode using `model.train()`.
   - A `running_loss` variable is initialized to track the cumulative loss over batches in the epoch.

3. **Batch Processing and Gradient Accumulation:**
   - The loop iterates over the `train_loader` with enumeration to keep track of batch indices.
   - Moves the batch's images and labels to the designated device.
   - Clears any existing gradients with `optimizer.zero_grad()`.
   - Computes model outputs for the current batch and calculates the loss.
   - Calls `loss.backward()` to compute the gradients, which are accumulated.
   - Every `grad_accum_steps` batches, the accumulated gradients are used to update the model parameters with `optimizer.step()`, and gradients are reset with another `optimizer.zero_grad()`.

4. **Learning Rate Scheduler:**
   - After processing all batches in an epoch, the scheduler adjusts the learning rate by calling `scheduler.step()`.

5. **Monitoring Training:**
   - Prints the average loss per batch at the end of each epoch to monitor training progress.

6. **Completion:**
   - Once all epochs are finished, the message "Training Complete!" is printed.


In [78]:
num_epochs = 50
grad_accum_steps = 2  # Accumulate gradients every 2 batches

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()

        # Update weights every `grad_accum_steps`
        if (i + 1) % grad_accum_steps == 0:
            optimizer.step()
            optimizer.zero_grad()

        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

print("Training Complete!")


Epoch 1/50, Loss: 2.1251
Epoch 2/50, Loss: 1.5063
Epoch 3/50, Loss: 1.2376
Epoch 4/50, Loss: 1.0258
Epoch 5/50, Loss: 0.9361
Epoch 6/50, Loss: 0.8637
Epoch 7/50, Loss: 0.7260
Epoch 8/50, Loss: 0.6833
Epoch 9/50, Loss: 0.5844
Epoch 10/50, Loss: 0.5009
Epoch 11/50, Loss: 0.4410
Epoch 12/50, Loss: 0.3983
Epoch 13/50, Loss: 0.3610
Epoch 14/50, Loss: 0.3411
Epoch 15/50, Loss: 0.2896
Epoch 16/50, Loss: 0.2970
Epoch 17/50, Loss: 0.2640
Epoch 18/50, Loss: 0.2804
Epoch 19/50, Loss: 0.2693
Epoch 20/50, Loss: 0.2555
Epoch 21/50, Loss: 0.2564
Epoch 22/50, Loss: 0.2796
Epoch 23/50, Loss: 0.2544
Epoch 24/50, Loss: 0.2539
Epoch 25/50, Loss: 0.2405
Epoch 26/50, Loss: 0.2389
Epoch 27/50, Loss: 0.2510
Epoch 28/50, Loss: 0.2187
Epoch 29/50, Loss: 0.2220
Epoch 30/50, Loss: 0.2125
Epoch 31/50, Loss: 0.2549
Epoch 32/50, Loss: 0.2831
Epoch 33/50, Loss: 0.2949
Epoch 34/50, Loss: 0.4248
Epoch 35/50, Loss: 0.4833
Epoch 36/50, Loss: 0.5070
Epoch 37/50, Loss: 0.4871
Epoch 38/50, Loss: 0.4222
Epoch 39/50, Loss: 0.

# Model Evaluation and Reporting

This snippet evaluates the trained model on the dataset using two key metrics: overall accuracy and a detailed classification report.

1. **Import Evaluation Metrics:**
   - `accuracy_score` calculates the overall accuracy.
   - `classification_report` generates a report including precision, recall, and F1-score for each class.

2. **Set Model to Evaluation Mode:**
   - `model.eval()` switches the model to evaluation mode, ensuring that layers like dropout and batch normalization operate in inference mode.

3. **Disable Gradient Calculations:**
   - The `torch.no_grad()` context is used to turn off gradient computation, speeding up the evaluation and reducing memory consumption.

4. **Inference Loop:**
   - Iterates through the `train_loader` to get batches of images and labels.
   - Moves images and labels to the computation device.
   - Computes outputs from the model and uses `torch.argmax` to determine the predicted class for each image.
   - Collects predictions and true labels, converting them to NumPy arrays for further processing.

5. **Accuracy Calculation:**
   - Computes the model's overall accuracy by comparing all predictions (`all_preds`) with the true labels (`all_labels`).

6. **Generate Detailed Classification Report:**
   - The report is generated with `classification_report` using the collected predictions and true labels.
   - `target_names` are specified using the keys from `train_dataset.label_mapping` to label each class in the report.


In [80]:
from sklearn.metrics import accuracy_score, classification_report

model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Compute Accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# Show Detailed Report
print(classification_report(all_labels, all_preds, target_names=train_dataset.label_mapping.keys()))


Model Accuracy: 92.24%
                                    precision    recall  f1-score   support

                              acne       0.89      0.92      0.91       129
                     acne-vulgaris       0.93      0.86      0.89       161
                 actinic-keratosis       0.85      0.97      0.91       144
              basal-cell-carcinoma       0.78      0.28      0.41       116
  basal-cell-carcinoma-morpheiform       0.96      1.00      0.98       135
                    dermatofibroma       0.94      1.00      0.97       137
                   dermatomyositis       0.97      0.99      0.98       121
                dyshidrotic-eczema       0.98      0.99      0.99       129
                            eczema       0.96      0.96      0.96       136
                   epidermal-nevus       0.96      1.00      0.98       157
                      folliculitis       0.90      0.88      0.89       155
                    kaposi-sarcoma       0.98      0.99      0.9

1. The overall accuracy is **92.24%**, indicating strong performance across classes.  
2. Most classes show high precision, recall, and F1-scores, often above **0.90**.  
3. The **support** column details the number of samples per class, confirming a balanced evaluation.  
4. The **macro avg** F1-score of **0.92** suggests the model performs well on all classes uniformly.  
5. The **weighted avg** F1-score of **0.92** indicates robust performance even in potentially imbalanced data.

# Making Predictions on Test Data and Saving Results



1. **Setup and Data Loading:**
   - Imports the necessary libraries (`pandas`, `os`, `PIL.Image`, and `torch`).
   - Retrieves the list of original class labels from the dataset's label mapping.
   - Defines the paths to the test images directory and the test CSV file that contains image metadata.

2. **Loading Test Data:**
   - Reads the test CSV file into a DataFrame (`test_df`), which contains the `md5hash` identifiers for each test image.

3. **Model Evaluation Preparation:**
   - Sets the model to evaluation mode using `model.eval()` to disable dropout and batch normalization updates.
   - Initializes an empty list (`predictions_list`) to store the prediction results.

4. **Prediction Loop:**
   - Iterates over each row in the test DataFrame.
   - For each image, constructs the file path using the `md5hash` value.
   - Checks if the image exists; if not, the loop skips to the next image.
   - Loads the image, converts it to RGB, and applies the same transformation pipeline used during training.
   - Adds a batch dimension to the image tensor and moves it to the selected device.
   - Feeds the preprocessed image


In [82]:
import pandas as pd
import os
from PIL import Image
import torch

# Define Class Labels (Map Back to Original Labels)
class_labels = list(train_dataset.label_mapping.keys())

# Path to Test Images & CSV
test_img_dir = "./extracted_files/test/test/"
test_csv_path = "./extracted_files/test.csv"
test_df = pd.read_csv(test_csv_path)

# Ensure model is in evaluation mode
model.eval()
predictions_list = []

# Make Predictions
with torch.no_grad():
    for index, row in test_df.iterrows():
        md5hash = row['md5hash']
        img_path = os.path.join(test_img_dir, md5hash + ".jpg")

        if not os.path.exists(img_path):
            continue  # Skip missing files

        # Load and Preprocess Image
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img).unsqueeze(0).to(device)

        # Get Model Prediction
        outputs = model(img_tensor)
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()
        predicted_label = class_labels[predicted_class]

        # Save Prediction
        predictions_list.append({"md5hash": md5hash, "label": predicted_label})

# Convert to DataFrame and Save as CSV
predictions_df = pd.DataFrame(predictions_list)
csv_output_path = "predictions_cleaned.csv"
predictions_df.to_csv(csv_output_path, index=False)

print(f" Predictions saved to: {csv_output_path}")


 Predictions saved to: predictions_cleaned.csv


In [66]:
import pandas as pd
import os
from PIL import Image
import torch

# Define Class Labels
class_labels = list(train_dataset.label_mapping.keys())

# Path to Test Images & CSV
test_img_dir = "./extracted_files/test/test/"
test_csv_path = "./extracted_files/test.csv"
test_df = pd.read_csv(test_csv_path)

# Ensure model is in eval mode
model.eval()
predictions_list = []

# Make Predictions
with torch.no_grad():
    for index, row in test_df.iterrows():
        md5hash = row['md5hash']
        img_path = os.path.join(test_img_dir, md5hash + ".jpg")

        if not os.path.exists(img_path):
            continue

        # Load and Preprocess Image
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img).unsqueeze(0).to(device)

        # Get Prediction
        outputs = model(img_tensor)
        probabilities = torch.nn.functional.softmax(outputs[0], dim=0)
        predicted_class = torch.argmax(probabilities).item()
        predicted_label = class_labels[predicted_class]

        # Save Prediction
        predictions_list.append({"md5hash": md5hash, "label": predicted_label})

# Convert to DataFrame and Save
predictions_df = pd.DataFrame(predictions_list)
csv_output_path = "predictions_cleaned.csv"
predictions_df.to_csv(csv_output_path, index=False)

print(f"Predictions saved to {csv_output_path} ")


Predictions saved to predictions_cleaned.csv 


NEW SUBMISSION USING DIFF MODEL : EfficientNet

In [10]:


import os
import zipfile
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
from sklearn.utils.class_weight import compute_class_weight
import timm

# =====================
# 1. Extract ZIP File
# =====================
zip_file_path = "bttai-ajl-2025.zip"
extract_dir = "extracted_files"
os.makedirs(extract_dir, exist_ok=True)
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)
print(f"Files extracted to: {extract_dir}")

# =====================
# 2. Device Setup
# =====================
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print("Using device:", device)

# =====================
# 3. Dataset Setup
# =====================
TRAIN_CSV = os.path.join(extract_dir, "train.csv")
TRAIN_IMG_DIR = os.path.join(extract_dir, "train", "train")

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

class SkinDiseaseDataset(Dataset):
    def __init__(self, csv_path, img_dir, transform=None):
        self.data = pd.read_csv(csv_path)
        self.img_dir = img_dir
        self.transform = transform
        self.label_mapping = {label: idx for idx, label in enumerate(sorted(self.data['label'].unique()))}
        self.data['label_encoded'] = self.data['label'].map(self.label_mapping)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        row = self.data.iloc[idx]
        label = torch.tensor(row['label_encoded'], dtype=torch.long)
        img_name = row['md5hash'] + ".jpg"
        img_path = os.path.join(self.img_dir, row['label'], img_name)
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        return image, label

train_dataset = SkinDiseaseDataset(TRAIN_CSV, TRAIN_IMG_DIR, transform=transform)
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# =====================
# 4. Model (EfficientNet)
# =====================
model = timm.create_model("efficientnet_b2", pretrained=True, num_classes=num_classes)
model.classifier = nn.Linear(model.classifier.in_features, len(train_dataset.label_mapping))
model = model.to(device).to(torch.float32)

# =====================
# 5. Loss & Optimizer
# =====================
labels = train_dataset.data["label"].values
class_weights = compute_class_weight("balanced", classes=np.unique(labels), y=labels)
class_weights = torch.tensor(class_weights, dtype=torch.float).to(device)

criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.AdamW(model.parameters(), lr=0.0005, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# =====================
# 6. Training Loop
# =====================
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device).float(), labels.to(device).long()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader):.4f}")

print("Training Complete!")

# =====================
# 7. Accuracy Evaluation
# =====================
from sklearn.metrics import accuracy_score
model.eval()
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in train_loader:
        images, labels = images.to(device).float(), labels.to(device)
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
print(f"Model Accuracy: {accuracy * 100:.2f}%")

# =====================
# 8. Generate Test Predictions CSV
# =====================
test_img_dir = os.path.join(extract_dir, "test", "test")
test_csv_path = os.path.join(extract_dir, "test.csv")
test_df = pd.read_csv(test_csv_path)
class_labels = list(train_dataset.label_mapping.keys())

model.eval()
predictions_list = []

with torch.no_grad():
    for index, row in test_df.iterrows():
        md5hash = row['md5hash']
        img_path = os.path.join(test_img_dir, md5hash + ".jpg")
        if not os.path.exists(img_path):
            continue
        img = Image.open(img_path).convert("RGB")
        img_tensor = transform(img).unsqueeze(0).to(device).float()
        outputs = model(img_tensor)
        predicted_class = torch.argmax(outputs, dim=1).item()
        predicted_label = class_labels[predicted_class]
        predictions_list.append({"md5hash": md5hash, "label": predicted_label})

predictions_df = pd.DataFrame(predictions_list)
predictions_df.to_csv("predictions_cleaned.csv", index=False)
print("Predictions saved to predictions_cleaned.csv")


Files extracted to: extracted_files
Using device: mps


model.safetensors:   0%|          | 0.00/21.4M [00:00<?, ?B/s]

Epoch 1/50, Loss: 2.2864
Epoch 2/50, Loss: 1.2556
Epoch 3/50, Loss: 0.7190
Epoch 4/50, Loss: 0.4533
Epoch 5/50, Loss: 0.3213
Epoch 6/50, Loss: 0.2610
Epoch 7/50, Loss: 0.2225
Epoch 8/50, Loss: 0.2072
Epoch 9/50, Loss: 0.1595
Epoch 10/50, Loss: 0.1157
Epoch 11/50, Loss: 0.0924
Epoch 12/50, Loss: 0.0424
Epoch 13/50, Loss: 0.0223
Epoch 14/50, Loss: 0.0213
Epoch 15/50, Loss: 0.0145
Epoch 16/50, Loss: 0.0205
Epoch 17/50, Loss: 0.0187
Epoch 18/50, Loss: 0.0135
Epoch 19/50, Loss: 0.0145
Epoch 20/50, Loss: 0.0158
Epoch 21/50, Loss: 0.0071
Epoch 22/50, Loss: 0.0074
Epoch 23/50, Loss: 0.0057
Epoch 24/50, Loss: 0.0049
Epoch 25/50, Loss: 0.0055
Epoch 26/50, Loss: 0.0064
Epoch 27/50, Loss: 0.0084
Epoch 28/50, Loss: 0.0063
Epoch 29/50, Loss: 0.0055
Epoch 30/50, Loss: 0.0080
Epoch 31/50, Loss: 0.0077
Epoch 32/50, Loss: 0.0049
Epoch 33/50, Loss: 0.0040
Epoch 34/50, Loss: 0.0056
Epoch 35/50, Loss: 0.0040
Epoch 36/50, Loss: 0.0047
Epoch 37/50, Loss: 0.0058
Epoch 38/50, Loss: 0.0057
Epoch 39/50, Loss: 0.